In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import pygmt
import pygplates

from gprm.datasets import Reconstructions, Geology, Rocks
from gprm.utils.spatial import topology_lookup
#from gprm.utils.create_gpml import gdf2gpml, gpml2gdf

#from collections import OrderedDict

import sys
sys.path.append('..')
import molchan

%load_ext autoreload
%autoreload 2


M2021 = Reconstructions.fetch_Merdith2021()

for f in M2021.continent_polygons[0]:
    if f.get_name() in ['Laurentia Parautochthon',
                        'Kootenay Terrane (NAM affinity Colpron et al 2007)',
                        'Cassiar Terrane (NAM affinity Colpron et al 2007)',
                        'Yukon-Tanana Terrane',
                        'Baikal-Muya']:
        f.set_valid_time(1000,-999)
        
        

In [ ]:
# Create a lookup table of the subduction zones reconstructed from topologies

subduction_lookup_time_step = 10.

sz_lookup = topology_lookup(M2021, 
                            np.arange(0,1001,subduction_lookup_time_step),
                            boundary_types=['subduction'])



In [ ]:
# Make two raster sequences, where:
# 1. Mask rasters where the pixels lying within continents (from those lying outside, therefore unreconstructable)
# 2. Distance rasters from subduction zone geometries

sz_distance_dict = [-180.,180.,-90.,90.]
sampling = 0.5

analysis_time_step = subduction_lookup_time_step

reconstruction_times = np.arange(0,1001,analysis_time_step)


reconstruction_raster_dict = molchan.generate_raster_sequence_from_polygons(
    M2021.continent_polygons[0],
    M2021.rotation_model,
    reconstruction_times,
    sampling=sampling
)

sz_distance_dict = molchan.generate_distance_raster_sequence(
    sz_lookup,
    M2021,
    reconstruction_times,
    sampling=sampling
)


In [ ]:
sz_distance_dict[200].plot()

In [ ]:
reconstruction_raster_dict[200.].plot()

In [ ]:
# Load the ore deposits, and reconstruct them to the time at the nearest time in the raster sequences

OreDeposits = Rocks.BaseMetalDeposits('PbZn-CD') 

OreDeposits_M2021 = M2021.assign_plate_ids(OreDeposits.query('Age<=1000'))

OreDeposits_M2021['AnalysisAge'] = np.round(OreDeposits_M2021['Age']/analysis_time_step)*analysis_time_step

Analysis_Result = M2021.reconstruct_to_time_of_appearance(OreDeposits_M2021, ReconstructTime='AnalysisAge')


In [ ]:
# Run experiments where the distance of each deposit to the nearest target at the time of deposit formation
# is determined from the distance raster sequence

# First combine the distance rasters with the mask rasters
sz_distance_dict_mask = molchan.combine_raster_sequences(sz_distance_dict, 
                                                           reconstruction_raster_dict)

# Now get the distances from the raster sequence mapped to each point
results_sz = molchan.space_time_distances(sz_distance_dict_mask,
                                            Analysis_Result,
                                            age_field_name='AnalysisAge')


In [ ]:
# To get the skill, we need to know not only the distances of the points to the features
# but also how these distances compare to the % of all points across all times within
# the same distance contour
sz_fractions = molchan.space_time_molchan_test(
    sz_distance_dict_mask, 
    results_sz['distance'],
    healpix_resolution=128)
print(sz_fractions[2])



plt.plot(sz_fractions[0],sz_fractions[1], 'r', 
         label='Cu-por to subduction zone  | Skill = {:0.3f}'.format(sz_fractions[2]))
plt.grid()
plt.legend()
plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel('Grid Fraction')
plt.ylabel('Fraction of points missed')

#plt.savefig('./images/molchan_space_time_1000Ma.png')
#plt.close()
plt.show()
